### Naver Rest API
- 파파고 번역 api
- 통합검색어 트렌드 api

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import requests

#### 0. request token 얻기
- `https://developers.naver.com`
- 1. Request Token 얻기 : 애플리케이션등록 -> app_key 획득
- 2. app_key를 이용해서 데이터 가져오기

In [5]:
client_id = '3cstgoTcPkvEieIVaeBF'
client_secret = 'iJQfzRpsz5'

#### 1. 파파고 번역 api
- `https://developers.naver.com/docs/papago/`
- 사용법
    - `https://developers.naver.com/docs/papago/papago-nmt-api-reference.md`

##### 1) Documnet > URL

In [3]:
url = 'https://openapi.naver.com/v1/papago/n2mt'
ko_txt = '데이터 크롤링은 돈이 됩니다.'
params = {
    'source' : 'ko',
    'target' : 'en',
    'text' : ko_txt,
}
params

{'source': 'ko', 'target': 'en', 'text': '데이터 크롤링은 돈이 됩니다.'}

In [6]:
headers = { 
    'Content-Type': 'application/json', 
    'X-Naver-Client-Id': client_id, 
    'X-Naver-Client-Secret': client_secret, 
}
headers

{'Content-Type': 'application/json',
 'X-Naver-Client-Id': '3cstgoTcPkvEieIVaeBF',
 'X-Naver-Client-Secret': 'iJQfzRpsz5'}

##### 2) Request > JSON(str)

In [35]:
# 인터넷에서 전달하는 데이터는 영문, 특수문자, 숫자만 사용가능
response = requests.post(url, params, headers=headers)
response

<Response [400]>

In [36]:
response.text

'{"errorCode":"-10001","errorMessage":"INVALID_REQUEST"}'

In [37]:
params

{'source': 'ko', 'target': 'en', 'text': '데이터 크롤링은 돈이 됩니다.'}

- json.dumps() 를 통해 한글문자 인코딩

In [38]:
import json
response = requests.post(url, json.dumps(params), headers=headers)
response

<Response [200]>

In [39]:
json.dumps(params)  # 한글을 인코딩 해줘야 함.

'{"source": "ko", "target": "en", "text": "\\ub370\\uc774\\ud130 \\ud06c\\ub864\\ub9c1\\uc740 \\ub3c8\\uc774 \\ub429\\ub2c8\\ub2e4."}'

In [40]:
response.text

'{"message":{"result":{"srcLangType":"ko","tarLangType":"en","translatedText":"Data crawling makes money.","engineType":"N2MT"},"@type":"response","@service":"naverservice.nmt.proxy","@version":"1.0.0"}}'

##### 3) JSON(str) > list, dir > text

In [41]:
en_txt = response.json()['message']['result']['translatedText']
en_txt

'Data crawling makes money.'

##### 4) Make function

In [19]:
# 4. make function 
def translate(ko_txt):
    client_id, client_secret = '3cstgoTcPkvEieIVaeBF', 'iJQfzRpsz5'
    url = 'https://openapi.naver.com/v1/papago/n2mt' 
    params = {'source': 'ko', 'target': 'en', 'text': ko_txt} 
    headers = { 'Content-Type': 'application/json', 
               'X-Naver-Client-Id': client_id, 
               'X-Naver-Client-Secret': client_secret, 
              } 
    response = requests.post(url, json.dumps(params), headers=headers) 
    return response.json()['message']['result']['translatedText']

In [20]:
translate('데이터사이언스는 유용합니다.')

'Data science is useful.'

##### 한글 엑셀 파일 영문으로 만들기

In [44]:
df = pd.read_excel('covid.xlsx')[['category', 'title']]
df.tail(2)

,category,title
4,104,"""코로나 확진자 '0명'인 날은 절대 오지 않는다"" 美전문가"
5,104,올림픽 다가오는데 도쿄 코로나 확산 빨라져...일주일새 25％↑


Series.apply(function)  : Series의 모든 데이터를 function에 적용한 결과를 출력하는 메소드

In [45]:
en_title = df['title'].apply(translate)
en_title

0    SK Bioscience Submits Phase 3 Trial Plan for C...
1    41 cumulative confirmed cases of COVID-19 at t...
2    New COVID-19 infections, 542 people by 9 p.m. ...
3    A confirmed case of COVID-19 occurred at a clu...
4    "The day when there are '0' confirmed cases of...
5    As the Olympics are approaching, the spread of...
Name: title, dtype: object

In [46]:
df['en_title'] = en_title
df.tail(2)

,category,title,en_title
4,104,"""코로나 확진자 '0명'인 날은 절대 오지 않는다"" 美전문가","""The day when there are '0' confirmed cases of..."
5,104,올림픽 다가오는데 도쿄 코로나 확산 빨라져...일주일새 25％↑,"As the Olympics are approaching, the spread of..."


In [51]:
df.to_excel('en_covid.xlsx', index=False, encoding='utf-8-sig')

#### 2. 통합검색어 트렌드 api
- 서비스 : `https://datalab.naver.com/keyword/trendSearch.naver`
- 내 애플리케이션 > dss 애플리케이션 > API 설정 > 데이터랩(검색어 트렌드) 추가
- 사용법 : `https://developers.naver.com/docs/serviceapi/datalab/search/search.md#통합-검색어-트렌드`